In [ ]:
%load_ext kamu

In [ ]:
%import_dataset ca.vancouver.opendata.property.parcel-polygons --alias lots

In [ ]:
print(lots.count())
lots.printSchema()

In [ ]:
%import_dataset ca.vancouver.opendata.property.tax-reports --alias tax

In [ ]:
print(tax.count())
tax.printSchema()

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW lot_tax AS (
SELECT
    t.*,
    l.*
FROM lots as l
JOIN tax as t
ON l.tax_coord = t.land_coordinate
WHERE
    t.legal_type = 'LAND'
    AND t.tax_assessment_year = 2019
    AND t.current_land_value is not null
)

In [ ]:
%%sql -o df -n 100000
SELECT
    tax_coord,
    ST_AsText(geometry) as geometry,
    current_land_value + current_improvement_value as current_total_value
FROM lot_tax

In [ ]:
%%local
import shapely.wkt
import geojson

def df_to_geojson(df, geom='geometry', props=None):
    if props is None:
        props = [
            c for c in df.columns
            if c != geom
        ]

    return geojson.FeatureCollection([
        geojson.Feature(
            geometry=shapely.wkt.loads(r[geom]),
            properties={p: r[p] for p in props}
        )
        for _, r in df.iterrows()
    ])

In [ ]:
%%local
gj = df_to_geojson(df)

In [ ]:
%%local
import os
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [ ]:
%%local
viz = ChoroplethViz(
    gj,
    style='mapbox://styles/mapbox/dark-v10',
    center=(-123.1207, 49.2827),
    zoom=10,
    access_token=token,
    color_property='current_total_value',
    color_stops=create_color_stops([1000000, 2000000, 3000000, 5000000, 10000000], colors='YlOrRd'),
    color_default='rgb(158,202,195)',
    line_width=0,
    opacity=1.0,
    legend_layout='horizontal',
    legend_key_shape='bar',
    legend_key_borders_on=False)

viz.show()